# Test closed complete is sufficient for non-flat complete in causal ABA


The current implementation of complete semantics doesn't check that the attacking subsets of assumptions are closed.
We argue that this does not affect complete extensions in the Causal ABA framework.
Independence fact-rules and the rules that derive independence from set of blocked path assumptions are what introduces the non-flatness into the ABAF. This notebook confirms that the attack and support sets before and after adding these rules remain the same, with the addition of the independences present in fact-rules.

This implies that the complete extensions are same because the attack and defence stays same. Defence stays same because the accepted set of assumptions can't attack the independence assumptions in facts anyways, as this will cause a conflict.

In [97]:
import sys
sys.path.append('../..')
sys.path.append('../../GradualABA')

In [98]:
from src.gradual.extra.abaf_factory_v0 import FactoryV0
from src.gradual.run import run_get_bsaf
from GradualABA.ABAF import ABAF
from src.utils.enums import Fact, RelationEnum
import src.causal_aba.assumptions as assums

In [99]:
def close_a_set(set_to_close: frozenset, all_supports: dict):
    """
    Closes a set by adding all assumptions that are supported by the set.
    TODO: iterate over rules, add whatever can be derived, untill a fixed point is reached.
    """
    new_set_to_close = set(set_to_close)
    for assumption, supports in all_supports.items():
        for support in supports:
            if set_to_close.issuperset(support):
                new_set_to_close.add(assumption)
    return frozenset(new_set_to_close)

def get_closed_attacks(all_attacks, all_supports):
    all_attacks_closed = dict()
    for target, attacks in all_attacks.items():
        new_attacks = set()
        for attack in attacks:
            new_attack = close_a_set(attack, all_supports)
            new_attacks.add(new_attack)
        all_attacks_closed[target] = set(new_attacks)
    return all_attacks_closed

In [100]:
class FactoryFlat(FactoryV0):
    @staticmethod
    def _add_independence_rules(solver, paths, X, Y, S):
        pass  # No rules with assumption in the head for flat ABAF

In [101]:
N_NODES=3

In [102]:
factory_flat = FactoryFlat(n_nodes=N_NODES)
bsaf_flat = run_get_bsaf(factory_flat,
                         facts=[],
                         abaf_class=ABAF)

factory_non_flat = FactoryV0(n_nodes=N_NODES)
bsaf_non_flat = run_get_bsaf(factory_non_flat,
                             facts=[],
                             abaf_class=ABAF)

Analysing rules: 100%|██████████| 108/108 [00:00<00:00, 287062.63it/s]


0.01s to build 267 arguments


Analysing rules: 100%|██████████| 114/114 [00:00<00:00, 787727.60it/s]

0.00s to build 273 arguments


In [103]:
get_closed_attacks(bsaf_flat.attacks, bsaf_flat.supports) == get_closed_attacks(bsaf_non_flat.attacks, bsaf_non_flat.supports)

True

In [104]:
bsaf_flat.attacks == get_closed_attacks(bsaf_flat.attacks, bsaf_flat.supports)

True

Confirms that attacks are same with or without the "independence <- all paths blocked" rule.

Now we proceed to confirm that attack sets change predictably, when independence facts are added. More specifically, the independence assumptions corresponding to these facts get appended to all the attacker sets.

In [105]:
class FactoryWithFacts(FactoryV0):
    @staticmethod
    def _add_fact(solver, fact: Fact):
        if fact.relation == RelationEnum.dep:
            # add dependency fact
            solver.add_rule(assums.contrary(assums.indep(fact.node1, fact.node2, fact.node_set)), [])
        elif fact.relation == RelationEnum.indep:
            # add independence fact
            solver.add_rule(assums.indep(fact.node1, fact.node2, fact.node_set), [])


In [106]:
factory_non_flat_w_facts = FactoryWithFacts(n_nodes=N_NODES)
solver_w_facts = factory_non_flat_w_facts.create_solver(facts=[])

fact = Fact(
    relation=RelationEnum.indep,
    node1=0,
    node2=1,
    node_set=set(),
    score=1
)

factory_non_flat_w_facts._add_fact(solver_w_facts, fact)

abaf_w_facts = solver_w_facts.get_abaf(abaf_class=ABAF)
bsaf_w_facts = abaf_w_facts.to_bsaf()


Analysing rules: 100%|██████████| 115/115 [00:00<00:00, 339201.80it/s]

0.01s to build 274 arguments


In [107]:
get_closed_attacks(bsaf_w_facts.attacks, bsaf_w_facts.supports) == get_closed_attacks(bsaf_flat.attacks, bsaf_flat.supports)

False

Let's confirm that the difference is the precense of the independence assumption in the fact

In [108]:
closed_attacks_w_facts = get_closed_attacks(bsaf_w_facts.attacks, bsaf_w_facts.supports)
closed_attacks_flat = get_closed_attacks(bsaf_flat.attacks, bsaf_flat.supports)

In [109]:
independence_assumption = solver_w_facts.name_to_assumption[assums.indep(0, 1, set())]
independence_assumption

Assumption(indep_0_1__, contrary=-indep_0_1__, weight=0.5)

In [110]:
closed_attacks_flat_added = dict()
for target, attacks in closed_attacks_flat.items():
    new_attacks = set()
    for attack in attacks:
        new_attack = frozenset({independence_assumption, *attack})
        new_attacks.add(new_attack)
    closed_attacks_flat_added[target] = new_attacks

In [111]:
closed_attacks_flat_added == closed_attacks_w_facts

True

This confirms our initial proposition that adding an independence fact is equivalend to adding the corresponding independence assumption to all attacking sets.

This means that complete extensions in Causal ABA can be achieved by considering not only closed attackers, but all attackers. Because closed attackers are same as all attackers with the addition of the independence assumptions from facts. However the accepted assumption set can't attack those independence assumptions (as it would cause a conflict) thus the attacked assumption sets are same, and consequently the defended assumptions are also same.